In [2]:
from skimage import io
from skimage import color
from skimage.filters import median
from skimage.morphology import disk
from skimage import exposure
from skimage.io import imsave, imread

import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
images = []

def preprocess(path):
    image = io.imread(path,0)
    
    #print(image.shape)
    #image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)


    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.medianBlur(image,3)
    
    
    hh,ww = image.shape[:2]
    thresh = cv2.threshold(image,0,255,cv2.THRESH_OTSU)[1] 
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    morph = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel)

    contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    big_contour = max(contours, key=cv2.contourArea)

    mask = np.zeros((hh,ww), dtype=np.uint8)
    cv2.drawContours(mask, [big_contour], 0, 255, cv2.FILLED)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (55,55))
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

    result = cv2.bitwise_and(image, image, mask=mask)

    clahe = cv2.createCLAHE(clipLimit = 5)
    clahe_img = clahe.apply(result) + 0
    #result = cv2.equalizeHist(image)
    

    images.append(clahe_img)
    #images.append(result)
   
    return clahe_img
    

In [42]:
ddsm_images = '/content/drive/MyDrive/TRIAL/normal_cases'
ddsm_preprocessed = '/content/drive/MyDrive/TRIAL/normal_new'
for root, directory, file in os.walk(ddsm_images):
  for f in file:
    
      ddsm_processed_img = preprocess(root + '/' + f)
      preprocess(root + '/' + f)
      ddsm_processed_img_path = ddsm_preprocessed + '/' + f
      #cv2.imwrite(ddsm_processed_img_path,255*ddsm_processed_img)
      imsave(ddsm_processed_img_path,ddsm_processed_img)
      print(ddsm_processed_img_path)

/content/drive/MyDrive/TRIAL/normal_new/Copy of 1.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 5.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 3.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 4.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 9.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 7.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 8.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 2.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 6.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 0.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 13.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 16.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 11.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 15.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 10.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 14.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 18.jpg
/content/drive/MyDrive/TRIAL/normal_new/Copy of 17.jpg
/content/drive/MyDri